In [92]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
import numpy as np

In [93]:
vfacts_df = pd.read_csv('vfacts_sales_df.csv', index_col=0)
abs_df = pd.read_csv('abs_df.csv', index_col=0)
abs_df = abs_df.astype({"Postcode": "string"}) # convert Postcode to string

Compare the national aggregation of ABS stock over time with the VFACTS sales over time (both in per month values)

In [94]:
from datetime import datetime
vfacts_year = [
    datetime.strptime(date, "%Y-%m").year + datetime.strptime(date, "%Y-%m").month / 12
    for date in vfacts_df.columns[3:]
]

Define the two and three parameter adoption curves

In [132]:
def adoption(t, a, b):
    return 1 / (1 + np.exp(-a * ((t-2020) - b)))

def adoption3(t, a, b, c):
   return c / (1 + np.exp(-a * ((t-2020) - b))) 

In [137]:
def get_calibrated_adoption_curve(tin, f, tout, calibration_points = [(2040,0.99)]):

    alpha0 = 0.5
    beta0 = 8
    gamma0 = 1.5

    p_guess = [alpha0, beta0]
    # p_guess3 = [alpha0, beta0, gamma0]

    t1 = tin.copy()
    f1 = f.copy()

    for p in calibration_points:
        t1.append(p[0])
        f1.append(p[1])

    p2_opt = curve_fit(f=adoption, xdata=t1, ydata=f1, p0=p_guess)
    alpha2 = p2_opt[0][0]
    beta2 = p2_opt[0][1]

    p_guess3 = [alpha2, beta2, 1]
    p3_opt = curve_fit(f=adoption3, xdata=t1, ydata=f1, p0=p_guess3)
    alpha3 = p3_opt[0][0]
    beta3 = p3_opt[0][1]
    gamma3 = p3_opt[0][2]

    return adoption(tout, alpha2, beta2), adoption3(tout, alpha3, beta3, gamma3)


Visualise the data and fit

In [145]:
def national_plot(fuel_type):
    # ABS stock data
    total_stock_change_per_year = (
        abs_df.groupby(["Fuel Type"]).sum().diff(axis=1).sum(axis=0)
    )
    fueltype_stock_change_per_year = (
        abs_df.groupby(["Fuel Type"]).sum().loc[fuel_type].diff()
    )  # change to vehicle stock per year
    fueltype_stock_change_per_year_percent = (
        fueltype_stock_change_per_year / total_stock_change_per_year * 100
    )

    # VFACTS sales data
    total_sales_per_year = vfacts_df.groupby(["Fuel Type"]).sum().sum(axis=0) * 12
    fueltype_sales_per_year = vfacts_df.groupby(["Fuel Type"]).sum().loc[fuel_type] * 12
    fueltype_sales_per_year_percent = (
        fueltype_sales_per_year / total_sales_per_year * 100
    )

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    x1 = fueltype_stock_change_per_year.index.astype(int).to_list()
    y1 = fueltype_stock_change_per_year.values.tolist()

    # Add traces
    fig.add_trace(
        go.Scatter(x=x1, y=y1, name="ABS Stock Change"),
        secondary_y=False,
    )

    x2 = vfacts_year
    y2 = fueltype_sales_per_year.values.tolist()

    fig.add_trace(
        go.Scatter(x=x2, y=y2, name="VFACTS Sales"),
        secondary_y=False,
    )

    x3 = fueltype_stock_change_per_year.index.astype(int).to_list()
    y3 = fueltype_stock_change_per_year_percent.values.tolist()

    fig.add_trace(
        go.Scatter(x=x3, y=y3, name="ABS Stock Change Percent"),
        secondary_y=True,
    )

    x4 = vfacts_year
    y4 = fueltype_sales_per_year_percent.values.tolist()

    fig.add_trace(
        go.Scatter(x=x4, y=y4, name="VFACTS Sales Percent"),
        secondary_y=True,
    )

    x5 = np.linspace(2020, 2040)
    calibration_points = [(2030, 0.7), (2040, 0.99)]
    y5, y6 = get_calibrated_adoption_curve(
        x4, [_y / 100 for _y in y4], x5, calibration_points=calibration_points
    )
    y5 = [_y * 100 for _y in y5]
    y6 = [_y * 100 for _y in y6]

    fig.add_trace(
        go.Scatter(
        x=[f[0] for f in calibration_points],
        y=[f[1] * 100 for f in calibration_points],
        name="Calibratoin Points",
        marker=dict(size=8, color="blue", symbol='circle-open'),
        mode="markers"),secondary_y=True
    )

    fig.add_trace(
        go.Scatter(x=x5, y=y5, name="Adoption Curve (2 par)"),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=x5, y=y6, name="Adoption Curve (3 par)"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="Sales & Stock Change Data for Fuel Type = " + fuel_type
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Year")

    # Set y-axes titles
    fig.update_yaxes(title_text="Vehicles Per Year", secondary_y=False)
    fig.update_yaxes(title_text="Percent of All Fuel Types", secondary_y=True)

    fig.show()


national_plot("Electric")


In [88]:
# select the state and vehicle type variables from those available in State and Vehicle Type columns

state = ''
vehicle_type = 'Passenger'

if state in vfacts_df['State'].unique():
    df = vfacts_df.loc[(vfacts_df['State'] == state)]
    df = df.drop(columns=['State'])
else:
    df = vfacts_df.groupby(['Vehicle Type', 'Fuel Type']).sum()
    df.reset_index(inplace=True)

if vehicle_type in df['Vehicle Type'].unique():
    df = df.loc[df['Vehicle Type'] == vehicle_type]
    df = df.drop(columns=['Vehicle Type'])
else:
    df = df.groupby(['Fuel Type']).sum()
    df.reset_index(inplace=True)

df = df.melt(
    id_vars=['Fuel Type'],
    var_name='Date',
    value_name='Sales Per Month',
)
total_sales = df.groupby(['Date']).sum()
percent_sales = [
    r['Sales Per Month'] / total_sales.loc[r['Date']]['Sales Per Month'] * 100
    for i, r in df.iterrows()
]
df['Percent Sales Per Month'] = percent_sales
df


,Fuel Type,Date,Sales Per Month,Percent Sales Per Month
0,Diesel,2020-09,388.0,2.189616
1,Electric,2020-09,96.0,0.541761
2,Hybrid,2020-09,1814.0,10.237020
3,Hydrogen,2020-09,0.0,0.000000
4,PHEV,2020-09,32.0,0.180587
5,Petrol,2020-09,15390.0,86.851016
6,Diesel,2020-12,338.0,1.691946
7,Electric,2020-12,66.0,0.330380
8,Hybrid,2020-12,2654.0,13.285278
9,Hydrogen,2020-12,0.0,0.000000


In [ ]:
abs_df

In [ ]:
national_passenger_df = df

In [ ]:
national_df = df

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

def plot_sales(df):
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig1 = px.area(df, x='Date', y='Sales Per Month', color='Fuel Type')
    for i in range(len(fig1['data'])):
        fig1['data'][i]['line']['width']=0
    fig2 = px.line(df, x='Date', y='Percent Sales Per Month', color='Fuel Type')
    fig2.update_traces(yaxis='y2')
    subplot_fig.add_traces(fig1.data + fig2.data)
    subplot_fig.show()

plot_sales(national_df)
plot_sales(national_passenger_df)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=75de7644-8bd4-4ecc-bdb2-2c9ef0ed94e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>